In [12]:
import os
os.chdir("C:/Users/opdar/BVM/Mtech/iiitb/2nd Sem/SPE/End_project/Object_Counting_1")

In [13]:
%pwd

'C:\\Users\\opdar\\BVM\\Mtech\\iiitb\\2nd Sem\\SPE\\End_project\\Object_Counting_1'

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    # params_is_augmentation: bool
    params_image_size: list
    best_model_path: Path

In [16]:
from src.Object_Counting.constants import *
from src.Object_Counting.utils.common import read_yaml, create_directories

In [17]:
from pathlib import Path

CONFIG_FILE_PATH = Path("C:/Users/opdar/BVM/Mtech/iiitb/2nd Sem/SPE/End_project/Object_Counting_1/config/config.yaml")
PARAMS_FILE_PATH = Path("C:/Users/opdar/BVM/Mtech/iiitb/2nd Sem/SPE/End_project/Object_Counting_1/params.yaml")

In [18]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            training_data=Path(training.data_yaml_path),
            base_model_path=Path(prepare_base_model.base_model_path),
            best_model_path=Path(training.best_model_path),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            # params_is_augmentation=params.AUGUMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [19]:
from ultralytics import YOLO
import shutil

In [20]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = YOLO(
            self.config.base_model_path
        )

    @staticmethod
    def save_model(path: Path, dest_path: Path):
        # model.save(path)
        shutil.copy(path, dest_path)


    def train(self):
        
        self.model.train(data=self.config.training_data, model="yolov8n.yaml",task="detect", \
                         epochs=self.config.params_epochs,workers=8,batch=self.config.params_batch_size,imgsz=self.config.params_image_size,\
                            project=self.config.root_dir,exist_ok=True)

        self.save_model(
            path=self.config.best_model_path,
            dest_path=self.config.trained_model_path
        )

In [21]:
import os
import yaml

def update_datasets_dir(path_to_script):
    # Determine the operating system
    is_windows = os.name == 'nt'
    
    # Define the YAML file path based on the operating system
    if is_windows:
        yaml_path = os.path.join(os.path.expanduser("~"), "AppData", "Roaming", "Ultralytics", "settings.yaml")
    else:
        yaml_path = os.path.join("/home/opdar", ".config", "Ultralytics", "settings.yaml")

    # Read the existing YAML file
    with open(yaml_path, 'r') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)

    # Update the datasets_dir key with the current script path
    # data['datasets_dir'] = path_to_script
    data.pop('datasets_dir', None)

    # Write the updated YAML file
    with open(yaml_path, 'w') as file:
        yaml.dump(data, file)

update_datasets_dir(os.getcwd())


In [22]:
%pwd

'C:\\Users\\opdar\\BVM\\Mtech\\iiitb\\2nd Sem\\SPE\\End_project\\Object_Counting_1'

In [24]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train()
    
except Exception as e:
    raise e


[2024-04-04 01:43:45,302: INFO: common: yaml file: C:\Users\opdar\BVM\Mtech\iiitb\2nd Sem\SPE\End_project\Object_Counting_1\config\config.yaml loaded successfully]
[2024-04-04 01:43:45,304: INFO: common: yaml file: C:\Users\opdar\BVM\Mtech\iiitb\2nd Sem\SPE\End_project\Object_Counting_1\params.yaml loaded successfully]
[2024-04-04 01:43:45,306: INFO: common: created directory at: artifacts]
[2024-04-04 01:43:45,308: INFO: common: created directory at: artifacts\training]
New https://pypi.org/project/ultralytics/8.1.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.9 🚀 Python-3.10.14 torch-2.2.2+cpu CPU (AMD Ryzen 5 6600H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=dataset.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=artifacts\training, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministi

train: Scanning C:\Users\opdar\BVM\Mtech\iiitb\2nd Sem\SPE\End_project\Object_Counting_1\artifacts\data_ingestion\dataset\train\labels.cache... 459 images, 0 backgrounds, 0 corrupt: 100%|██████████| 459/459 [00:00<?, ?it/s]
val: Scanning C:\Users\opdar\BVM\Mtech\iiitb\2nd Sem\SPE\End_project\Object_Counting_1\artifacts\data_ingestion\dataset\valid\labels.cache... 90 images, 0 backgrounds, 0 corrupt: 100%|██████████| 90/90 [00:00<?, ?it/s]


Plotting labels to artifacts\training\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to artifacts\training\train
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      1.527      2.967      1.026        142        320: 100%|██████████| 29/29 [00:42<00:00,  1.47s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.40s/it]

                   all         90       1502      0.964     0.0649      0.109     0.0575


...

      10/10         0G      1.077     0.7171     0.8749        145        320: 100%|██████████| 29/29 [00:39<00:00,  1.38s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.63s/it]

                   all         90       1502       0.53       0.37       0.34      0.202



10 epochs completed in 0.127 hours.
Optimizer stripped from artifacts\training\train\weights\last.pt, 6.2MB
Optimizer stripped from artifacts\training\train\weights\best.pt, 6.2MB

Validating artifacts\training\train\weights\best.pt...
Ultralytics YOLOv8.1.9 🚀 Python-3.10.14 torch-2.2.2+cpu CPU (AMD Ryzen 5 6600H with Radeon Graphics)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.35s/it]


                   all         90       1502      0.535      0.365      0.341      0.202
               bicycle         90         30      0.142     0.0667     0.0709     0.0308
                   bus         90          4          1          0          0          0
                   car         90       1030      0.715      0.898      0.897      0.571
             motorbike         90        400      0.492       0.33       0.31      0.136
                person         90         38      0.325      0.532      0.426      0.272
Speed: 0.7ms preprocess, 27.9ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to artifacts\training\train
